In [1]:
import numpy as np
import pandas as pd 
#import matplotlib.pyplot as plt
#import seaborn as sns
pd.set_option('max_columns', 50)
####model libraries ####################
from sklearn.linear_model import Ridge, Lasso,SGDRegressor,LinearRegression
from xgboost import XGBRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor,AdaBoostRegressor,RandomForestRegressor, ExtraTreesRegressor
from sklearn.tree import ExtraTreeRegressor,DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
#########################################
from sklearn.cross_decomposition import PLSRegression

from sklearn.metrics import mean_squared_error, mean_absolute_error,make_scorer,r2_score,mean_absolute_percentage_error,explained_variance_score
from sklearn.preprocessing import MinMaxScaler

#### visulization #####
import plotly.graph_objects as go



In [2]:
#df = pd.ExcelFile('C:/Users/halil/Desktop/soalr_data.xlsx').parse('sheet 1')
weather_raw = pd.read_excel('C:/Users/halil/Desktop/soalr_data.xlsx',sheet_name="weather")
guneko_raw = pd.read_excel('C:/Users/halil/Desktop/soalr_data.xlsx',sheet_name="1000255-GUNEKO")

In [3]:
guneko_production = guneko_raw[["Date","Production"]]
guneko_gti = guneko_raw[["Date.1","GTI"]]


weather_guneko = weather_raw.loc[weather_raw.name ==1000255]
ali = pd.merge(guneko_production,weather_guneko,left_on="Date",right_on="date")
dataset = ali.drop(["name","date","lat","lon"],axis=1)

In [4]:
conversion = dataset["Date"].astype(str)

dataset["year"] = conversion.apply(lambda x:int(x[:4]))
dataset["month"] = conversion.apply(lambda x:int(x[5:7]))
dataset["day"] = conversion.apply(lambda x:int(x[8:10]))
dataset["hour"] = conversion.apply(lambda x:int(x[11:13]))
onehot_encoded = dataset.drop(["Date"],axis=1)

In [ ]:

def metrics(y_train1,y_train_pred1,y_test1,y_test_pred1):
    print("for train data, mean squared error is",mean_squared_error(y_train1, y_train_pred1))
    print("for train data, mean absolute error is:",mean_absolute_error(y_train1, y_train_pred1))
    print("for train data, mean absolute percentage error is",mean_absolute_percentage_error(y_train1, y_train_pred1),"\n")

    print("for test data, mean squared error is",mean_squared_error(y_test1, y_test_pred1))
    print("for test data, mean absolute error is:",mean_absolute_error(y_test1, y_test_pred1))
    print("for test data, mean absolute percentage error is",mean_absolute_percentage_error(y_test1, y_test_pred1),"\n")


In [5]:
#normalization in range (-1,1)
def normalize(df,column_list):
    for col in column_list:
        feature_range = (0,1)
        min_max_scaler = MinMaxScaler(feature_range=feature_range)

        df[col] = min_max_scaler.fit_transform(df[col].values.reshape(-1,1))

    return df

In [20]:
def cal_model_error(x_train,x_test,y_train,y_test,model_name):
    model = model_name() 
    model.fit(x_train,y_train)
    train_prediction = model.predict(x_train)
    test_prediction = model.predict(x_test)

    return round(mean_squared_error(y_train, train_prediction),9), round(mean_squared_error(y_test, test_prediction),9)


In [ ]:
"""

def calculate_ridge(x_train,x_test,y_train,y_test):
    model = Ridge(alpha=1) 
    model.fit(x_train,y_train)
    train_prediction = model.predict(x_train)
    test_prediction = model.predict(x_test)

    return mean_squared_error(y_train, train_prediction), mean_squared_error(y_test, test_prediction)



def calculate_lasso(x_train,x_test,y_train,y_test):
    model = Lasso(alpha=1) 
    model.fit(x_train,y_train)
    train_prediction = model.predict(x_train)
    test_prediction = model.predict(x_test)

    return mean_squared_error(y_train, train_prediction), mean_squared_error(y_test, test_prediction)

    
def calculate_linear_reg(x_train,x_test,y_train,y_test):

    model = LinearRegression()
    model.fit(x_train, y_train)
    train_prediction = model.predict(x_train)
    test_prediction = model.predict(x_test)
    return mean_squared_error(y_train, train_prediction), mean_squared_error(y_test, test_prediction)

def calculate_xgb(x_train,x_test,y_train,y_test):
    model = XGBRegressor()
    model.fit(x_train, y_train)
    train_prediction = model.predict(x_train)
    test_prediction = model.predict(x_test)
    return mean_squared_error(y_train, train_prediction), mean_squared_error(y_test, test_prediction)

def calculate_kneighbors(x_train,x_test,y_train,y_test):
    model = KNeighborsRegressor()
    model.fit(x_train, y_train)
    train_prediction = model.predict(x_train)
    test_prediction = model.predict(x_test)
    return mean_squared_error(y_train, train_prediction), mean_squared_error(y_test, test_prediction)

def calculate_decisiontree(x_train,x_test,y_train,y_test):
    model = DecisionTreeRegressor()
    model.fit(x_train, y_train)
    train_prediction = model.predict(x_train)
    test_prediction = model.predict(x_test)
    return mean_squared_error(y_train, train_prediction), mean_squared_error(y_test, test_prediction)

def calculate_randomforest(x_train,x_test,y_train,y_test):
    model = RandomForestRegressor()
    model.fit(x_train, y_train)
    train_prediction = model.predict(x_train)
    test_prediction = model.predict(x_test)
    return mean_squared_error(y_train, train_prediction), mean_squared_error(y_test, test_prediction)
"""


In [7]:
def make_data(dataframe):
    train = dataframe[:len(dataframe)- 24*5]
    test = dataframe[len(dataframe)- 24*5:]

    x_train = train.copy()
    x_test = test.copy()

    y_train = x_train.pop('Production')
    y_test = x_test.pop('Production')

    return x_train, x_test, y_train, y_test 

In [8]:
def get_year(whole_data,target_year):

    whole_data = whole_data.loc[whole_data.year== target_year]
    whole_data = whole_data.drop(["year"],axis=1)
    whole_data = whole_data.reset_index(drop=True)

    return whole_data

In [9]:
def get_months(target_data):
    months = []

    for i in range(1,13):
        month_data = target_data.loc[target_data["month"]==i].copy()
        aw1 = month_data.reset_index(drop=True)
        aw1 = aw1.drop(["month"],axis=1)
        months.append(aw1)

    return months

In [ ]:
#x_train, x_test, y_train, y_test = make_data(data_months[0])

In [10]:
#error calculate method with only one function , dynamically but can give error 

def calculate_errors(month_data):
    errors = []
    ridge,lasso,decisiontree,kneighbors,linear,randomforest,xgb,adaboost,neuralnetwork= 0,0,0,0,0,0,0,0,0
    model_list = [ridge,lasso,decisiontree,kneighbors,linear,randomforest,xgb,adaboost,neuralnetwork]

    models = [Ridge,Lasso,DecisionTreeRegressor,KNeighborsRegressor,LinearRegression,RandomForestRegressor,XGBRegressor,AdaBoostRegressor,MLPRegressor,MLPRegressor]
    
    for i in range(0,12):
        x_train, x_test, y_train, y_test = make_data(month_data[i])

        
        for m in range(0,9):
            model_list[m] = cal_model_error(x_train, x_test, y_train, y_test,models[m])

        errors.append({f"ridge":model_list[0],"lasso":model_list[1],"decisiontree":model_list[2],"kneighbors":model_list[3],
        "linear":model_list[4],"randomforest":model_list[5],"xgb":model_list[6],"adaboost":model_list[7],"neuralnetwork":model_list[8]})
    return errors

In [ ]:
#error calculation method that passing different functions for each model
"""def calculate_errors(month_data):
    errors = []
    for i in range(0,12):
        x_train, x_test, y_train, y_test = make_data(month_data[i])

        ridge = calculate_ridge(x_train, x_test, y_train, y_test)
        lasso = calculate_lasso(x_train, x_test, y_train, y_test)
        decisiontree = calculate_decisiontree(x_train, x_test, y_train, y_test)
        kneighbors = calculate_kneighbors(x_train, x_test, y_train, y_test)
        linear= calculate_linear_reg(x_train, x_test, y_train, y_test)
        randomforest = calculate_randomforest(x_train, x_test, y_train, y_test)
        xgb =calculate_xgb(x_train, x_test, y_train, y_test)

        errors.append({f"ridge":ridge,"lasso":lasso,"decisiontree":decisiontree,
                "kneighbors":kneighbors,"linear":linear,"randomforest":randomforest,"xgb":xgb})
    return errors"""

In [12]:
def dataframe_maker(errors):    
    aylar = ["January","February","March","April","May","June","July","August","September","October","November","December"]

    ridge_train, ridge_test = [] , []
    lasso_train, lasso_test= [], []
    decisiontree_train, decisiontree_test = [], []
    kneighbors_train, kneighbors_test = [], []
    linear_train, linear_test = [], []
    randomforest_train, randomforest_test = [], []
    xgb_train, xgb_test = [], []
    adaboost_train, adaboost_test = [], []
    neuralnetwork_train, neuralnetwork_test = [], []
    for i in errors:
        ridge_train.append(i["ridge"][0])
        ridge_test.append(i["ridge"][1])

        lasso_train.append(i["lasso"][0])
        lasso_test.append(i["lasso"][1])

        decisiontree_train.append(i["decisiontree"][0])
        decisiontree_test.append(i["decisiontree"][1])

        kneighbors_train.append(i["kneighbors"][0])
        kneighbors_test.append(i["kneighbors"][1])

        linear_train.append(i["linear"][0])
        linear_test.append(i["linear"][1])

        randomforest_train.append(i["randomforest"][0])
        randomforest_test.append(i["randomforest"][1])
        
        xgb_train.append(i["xgb"][0])
        xgb_test.append(i["xgb"][1])

        adaboost_train.append(i["adaboost"][0])
        adaboost_test.append(i["adaboost"][1])

        neuralnetwork_train.append(i["neuralnetwork"][0])
        neuralnetwork_test.append(i["neuralnetwork"][1])
        

    seri5 = pd.Series(aylar)
    train_frames = {"months":seri5,"ridge":ridge_train,"lasso":lasso_train,"decisiontree":decisiontree_train,"kneighbors":kneighbors_train,
                    "linear":linear_train,"randomforest":randomforest_train,"xgb": xgb_train,"adaboost":adaboost_train,"neuralnetwork": neuralnetwork_train}      

    test_frames = {"months":seri5,"ridge":ridge_test,"lasso":lasso_test,"decisiontree":decisiontree_test,"kneighbors":kneighbors_test,
                    "linear":linear_test,"randomforest":randomforest_test,"xgb": xgb_test,"adaboost":adaboost_test,"neuralnetwork":neuralnetwork_test}  

    train_dataframe = pd.DataFrame(train_frames)
    test_dataframe = pd.DataFrame(test_frames)
    return train_dataframe, test_dataframe    

In [34]:
def plot_resulst(train, test, model_name):
    
    fig = go.Figure(layout_yaxis_range=[0,0.05])
    fig.add_trace(go.Bar(x=test["months"],y=train[model_name],name=f'{model_name} Model Train Results', marker_color='indianred'))

    fig.add_trace(go.Bar(x=test["months"], y=test[model_name], name=f'{model_name} Model Test Results',marker_color='lightsalmon'))

    # Here we modify the tickangle of the xaxis, resulting in rotated labels.
    fig.update_layout(barmode='group', xaxis_tickangle=-45,title=f"{model_name} results")
    fig.show()

In [21]:
normalized = normalize(onehot_encoded,onehot_encoded.columns[0:11])
data_2021 = get_year(normalized,2021)


In [22]:
data_months = get_months(data_2021)
errors = calculate_errors(data_months)
train_errors,test_errors = dataframe_maker(errors)

In [ ]:
plot_resulst(train_errors,test_errors,"ridge")

In [23]:
train_errors

,months,ridge,lasso,decisiontree,kneighbors,linear,randomforest,xgb,adaboost,neuralnetwork
0,January,0.008753,0.041575,0.0,0.002099,0.008693,0.000684,6.790000e-07,0.004892,0.009879
1,February,0.011076,0.059925,0.0,0.003674,0.010661,0.000813,8.140000e-07,0.007039,0.012254
2,March,0.009709,0.066898,0.0,0.004217,0.008829,0.000806,8.360000e-07,0.005659,0.009335
3,April,0.014515,0.095987,0.0,0.007148,0.013398,0.001323,1.036000e-06,0.008581,0.013876
4,May,0.009498,0.135633,0.0,0.003058,0.009158,0.000650,1.258000e-06,0.005800,0.006757
5,June,0.009837,0.138775,0.0,0.002834,0.009231,0.000846,1.224000e-06,0.004612,0.012478
6,July,0.017091,0.119389,0.0,0.003406,0.016519,0.001189,1.150000e-06,0.011377,0.012745
7,August,0.012336,0.117205,0.0,0.003384,0.011912,0.000841,1.068000e-06,0.006856,0.009137
8,September,0.008508,0.123629,0.0,0.001490,0.008227,0.000551,1.034000e-06,0.003757,0.009112
9,October,0.007902,0.111887,0.0,0.001603,0.007744,0.000427,9.470000e-07,0.002598,0.008948


In [18]:
train_errors

,months,ridge,lasso,decisiontree,kneighbors,linear,randomforest,xgb,adaboost,neuralnetwork
0,January,0.00875,0.04158,0.0,0.00210,0.00869,0.00075,0.0,0.00354,0.00878
1,February,0.01108,0.05993,0.0,0.00367,0.01066,0.00074,0.0,0.00505,0.01423
2,March,0.00971,0.06690,0.0,0.00422,0.00883,0.00076,0.0,0.00621,0.01175
3,April,0.01451,0.09599,0.0,0.00715,0.01340,0.00137,0.0,0.00937,0.01395
4,May,0.00950,0.13563,0.0,0.00306,0.00916,0.00067,0.0,0.00574,0.00827
5,June,0.00984,0.13877,0.0,0.00283,0.00923,0.00078,0.0,0.00495,0.01466
6,July,0.01709,0.11939,0.0,0.00341,0.01652,0.00124,0.0,0.01254,0.02126
7,August,0.01234,0.11720,0.0,0.00338,0.01191,0.00100,0.0,0.00757,0.01201
8,September,0.00851,0.12363,0.0,0.00149,0.00823,0.00055,0.0,0.00350,0.00451
9,October,0.00790,0.11189,0.0,0.00160,0.00774,0.00047,0.0,0.00329,0.01161


In [19]:
test_errors

,months,ridge,lasso,decisiontree,kneighbors,linear,randomforest,xgb,adaboost,neuralnetwork
0,January,0.00679,0.02970,0.01042,0.02042,0.00667,0.00640,0.00766,0.00624,0.01410
1,February,0.00628,0.10562,0.00201,0.00248,0.00623,0.00140,0.00167,0.00572,0.02041
2,March,0.01646,0.11671,0.02492,0.10646,0.01567,0.01680,0.01697,0.01900,0.03309
3,April,0.02178,0.10979,0.02324,0.03239,0.02427,0.01692,0.01804,0.02483,0.03452
4,May,0.01220,0.12570,0.01962,0.01532,0.01203,0.01154,0.01407,0.01308,0.02423
5,June,0.01959,0.11778,0.02018,0.01645,0.01867,0.01688,0.01647,0.02386,0.02234
6,July,0.00729,0.13275,0.00132,0.00267,0.00660,0.00145,0.00129,0.01578,0.03331
7,August,0.00576,0.12201,0.00972,0.00271,0.00617,0.00134,0.00304,0.00789,0.01090
8,September,0.00780,0.12989,0.00393,0.00284,0.00734,0.00149,0.00096,0.00484,0.01311
9,October,0.01747,0.06940,0.01545,0.01513,0.05919,0.01256,0.01167,0.01517,0.03282


In [35]:
plot_resulst(train_errors,test_errors,"ridge")

In [36]:
plot_resulst(train_errors,test_errors,"lasso")

In [37]:
plot_resulst(train_errors,test_errors,"decisiontree")

In [38]:
plot_resulst(train_errors,test_errors,"kneighbors")

In [39]:
plot_resulst(train_errors,test_errors,"linear")

In [40]:
plot_resulst(train_errors,test_errors,"randomforest")

In [41]:
plot_resulst(train_errors,test_errors,"xgb")

In [42]:
plot_resulst(train_errors,test_errors,"adaboost")

In [43]:
plot_resulst(train_errors,test_errors,"neuralnetwork")